In [1]:
import h5py
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
path = "./processed-data/TSFresh/selected_features_all.csv"
selected_features = pd.read_csv(path, header='infer')
selected_features = selected_features.values

In [3]:
with h5py.File('./human-activity/train.h5', 'r') as f:
  y = f['y'][()]

target = pd.DataFrame(y, columns=['target'])
target  = target["target"]

In [4]:
seed = 42
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, test_size=0.2, random_state=seed)

In [5]:
y_train = y_train.values

# SVM

In [6]:
from models.svm import *

In [7]:
import time
from sklearn.decomposition import PCA

num_classes = len(np.unique(y))
C = 1.0
alpha = 0.01
epochs = 5000

np.random.seed(2024)

# Crea un archivo CSV y escribe el encabezado
with open('accuracy_results_svm.csv', 'w') as f:
    f.write('n_components,accuracy,time\n')

# Varia el número de componentes desde 10 hasta 100 de 10 en 10
for n_components in range(10, 101, 10):
    start_time = time.time()
    
    # Aplica PCA
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Entrena el clasificador
    classifier = training_multiclass(X_train_pca, y_train, C, alpha, epochs, num_classes)
    Y_pred = testing(X_test_pca, classifier)

    accuracy = sum(Y_pred == y_test) / len(y_test) * 100

    # Calcula el tiempo de ejecución
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Guarda los resultados en el archivo CSV
    with open('accuracy_results_svm.csv', 'a') as f:
        f.write(f'{n_components},{accuracy},{elapsed_time}\n')

    print(f'Componentes: {n_components}, Accuracy: {accuracy:.2f}%, Tiempo: {elapsed_time:.2f} segundos')

print("Resultados guardados en accuracy_results.csv")

Componentes: 10, Accuracy: 22.50%, Tiempo: 28.54 segundos
Componentes: 20, Accuracy: 23.32%, Tiempo: 34.73 segundos
Componentes: 30, Accuracy: 27.67%, Tiempo: 65.08 segundos
Componentes: 40, Accuracy: 23.25%, Tiempo: 96.70 segundos
Componentes: 50, Accuracy: 16.32%, Tiempo: 79.20 segundos
Componentes: 60, Accuracy: 24.00%, Tiempo: 92.87 segundos
Componentes: 70, Accuracy: 24.47%, Tiempo: 131.97 segundos
Componentes: 80, Accuracy: 24.07%, Tiempo: 107.90 segundos
Componentes: 90, Accuracy: 23.59%, Tiempo: 102.65 segundos
Componentes: 100, Accuracy: 23.11%, Tiempo: 4103.78 segundos
Resultados guardados en accuracy_results.csv
